**Эту тетрадь сохранил до того как начал все оборачивать в функции**

Важные моменты:
1) +Убрать название заказчика из названий от дилеров
2) +Проверить названия на всех платформах, и брать все уникальные слова, что есть на платформах (по каждому названию в таблице продукции)
3) +Соединять данные по последней таблице разметки, с нее же делить на выборки
4) ?2 варианта: обучать по таблице с разметкой (как? Собрать свою метрику Андрей предлаал) или без учителя (сравнивать с ними же, но диллеров)
5) Выдавать не чисто 5 вариантов, а те варианты, которые проходят какой то порог, либо сделать настраиваемый. (В ТЗ указано, что это должно быть настраиваемым, значит первыми выводить наиболее вероятные)
6) -Пайплайн или нет? Возможно
7) -Подходит ли для задачи BERT? Андрей пробует
8) -Умеет ли кто работать с классами? 
9) -Градиентный бустинг в какой момент? 

Работа с текстом:
0) +очистка от знаков, один регистр
1) +Лемматизация
2) Мешок слов
3) N-граммы
4) +TF-IDF



Ход работы:

0) Формат вывода для бэкэнда (допустим передаем лучшие 10 в порядке убывания, а пользователь через интерфейс сам выбирает сколько выводить)
1) +Очистка, преобразование текста, Собрать уникальные слова в названиях (от заказчика)
2) ?собрать датасет для МО
3) +Утвердить или разработать метрику оценки
4) проверить разные стратегии: обучить или по наименьшему расстояни между векторами
5) Сравнить стратегии/модели
6) Завернуть в функцию/ функции

Ира:
1) +Изменение чисел на строки, очистка от прочих знаков
2) +Убрать просепт в названиях
3) +Лемматизация текста
4) пайплайн на будущее

Денис:
1) +Регулярное выражение на изменение единиц измерения объема, выделить их в отдельную фичу
2) +Собрать уникальные слова с названий всех платформ
3) +Выдача первого результата если есть артикул

Андрей:
1) пробует BERT
2) TF-IDF

In [1]:
!pip install pandas
!pip install numpy
!pip install spacy
!pip install faiss-cpu
!pip install nltk
!pip install scikit-learn

In [2]:
import numpy as np
import matplotlib
import pandas as pd
import re
from matplotlib import pyplot as plt
import spacy
from collections import OrderedDict

from scipy.spatial.distance import pdist, squareform


import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from nltk.corpus import stopwords as nltk_stopwords
import faiss

In [5]:
dealers = pd.read_csv('prosept/marketing_dealer.csv', sep=';').sort_values('id').reset_index(drop=True) #Сортируем диллеров по id 
parser = pd.read_csv('prosept/marketing_dealerprice.csv', sep=';').drop(['id','product_url', 'date'], axis=1).drop_duplicates().rename(columns={'product_key': 'key'}).reset_index(drop=True)
products = pd.read_csv('prosept/marketing_product.csv', sep=';').drop(['Unnamed: 0', 'ean_13',
                                                                'category_id', 'ozon_article', 'wb_article',
                                                                'ym_article', 'wb_article_td'], axis=1)
matches = pd.read_csv('prosept/marketing_productdealerkey.csv', sep=';').drop(['id','dealer_id'], axis=1)

**Функции**

In [6]:
def text_worker(name): #отдельно функция для предварительной очистки текста
    #добавляем пробел на явных стыках и переходах
    pattern = [r"([а-я])([a-zA-ZА-Я])", r"([А-Я])([A-Za-z])", r"([a-z])([A-Zа-яА-Я])"]     
    for p in pattern:
        try:
            name = re.sub(p, "\\1 \\2", name)
        except:
            pass
    name = name.lower()
    bad = ['просепт','prosept50','prosept50,', 'prosepteco50','prosepteco50,','ultra', 'crystal', '-ая', 'prosept','ф/п']
    good = ['prosept',' prosept50 ',' prosept50 ', ' prosepteco50 ', ' prosepteco50 ', ' ultra ', ' crystal ', '', '', 'пакет']
    for o,n in zip(bad, good):
        name = name.replace(o,n)
    return name

def change_equal(name): 
    '''Функция принимает текст и пытается вытащить из него 
    артитул, еденицы измерения, количество (переводя в одну единицу измерения), текст очищеный от предыдущих элементов
    и все это возвращает в види словаря'''
    name = text_worker(name)
    dictionary={}
    try:
        article = re.search(r' \d+-\d+/?\d?[а-я]?',name)[0].strip()
        name = name.replace(article,'')
    except:
        article = np.nan
    try:
        v = re.findall(r'\d+(?:[\.,]\d+)? ?(?:мл|кг|г|л|шт)',name)[-1]
        dimension = re.search(r'[млкгшт]+',v)[0]
        quantity = float(v.replace(dimension,'').replace(',','.'))
        if dimension == 'мл':
            quantity = quantity/1000
            dimension = 'л'
        elif dimension == 'г':
            quantity = quantity/1000
            dimension = 'кг'
        name = name.replace(v,'')
        #for _v_ in v
        #    name = name.replace(_v_,'')
    except:
        dimension = np.nan
        quantity = 0
    finally:
        dictionary['name_new'] = ' '.join(re.sub(r'\W+', ' ', name).split(r'\W+'))
        dictionary['article'] = article
        dictionary['dimension'] = dimension
        dictionary['quantity'] = quantity
        return dictionary        

In [7]:
parser_new = parser['product_name'].apply(lambda x: change_equal(x)).apply(pd.Series)

In [8]:
parser_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name_new   3001 non-null   object 
 1   article    883 non-null    object 
 2   dimension  2837 non-null   object 
 3   quantity   3001 non-null   float64
dtypes: float64(1), object(3)
memory usage: 93.9+ KB


In [10]:
parser_final = pd.concat([parser, parser_new],axis=1).drop(['product_name'], axis=1)
parser_final['article'] = parser_final['article'].fillna('')
parser_final

,key,price,dealer_id,name_new,article,dimension,quantity
0,546227,233.0,2,средство универсальное universal spray,,л,0.5
1,546408,175.0,2,концентрат multipower для мытья полов цитрус,,л,1.0
2,546234,285.0,2,средство для чистки люстр universal anti dust,,л,0.5
3,651258,362.0,2,удалитель ржавчины rust remover,023-05,л,0.5
4,546355,205.0,2,средство моющее для бани и сауны multipower wood,,л,1.0
...,...,...,...,...,...,...,...
2996,717999545,1350.0,17,строительный антисептик глубокого проникновения,,кг,9.9
2997,718000129,1282.0,17,огнебиозащита для древесины огнебио prof 1 выс...,,кг,12.0
2998,900996549,294.0,18,отбеливатель для древесины eco 50 готовый состав,,кг,1.0
2999,530170161,751.0,18,строительный антисептик,,кг,5.6


In [200]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   article            496 non-null    object 
 2   name               496 non-null    object 
 3   cost               491 non-null    float64
 4   recommended_price  491 non-null    float64
 5   ozon_name          496 non-null    object 
 6   name_1c            496 non-null    object 
 7   wb_name            496 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 31.1+ KB


In [184]:
parser_final.isna().sum()

key            0
price          0
dealer_id      0
name_new       0
article        0
dimension    164
quantity       0
dtype: int64

In [11]:
#заполняем пропуски
name_columns = ['name_1c', 'ozon_name', 'name', 'wb_name']
products['name_1c'] = products['name_1c'].fillna(products['name'])
products['ozon_name'] = products['ozon_name'].fillna(products['name_1c'])
products[name_columns] = products[name_columns].fillna('')

In [12]:
#Из субъективно лучшей колонки названия
name_1c = products['name_1c'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'name_1c_new'})['name_1c_new']
ozon_name = products['ozon_name'].apply(change_equal).apply(pd.Series).drop(['article'],axis=1).rename(columns={'name_new': 'ozon_name_new'})
name_1 = products['name'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'name_new'})['name_new']
wb_name = products['wb_name'].apply(change_equal).apply(pd.Series).rename(columns={'name_new': 'wb_name_new'})['wb_name_new']
products1 = pd.concat([products, ozon_name, name_1c, wb_name, name_1],axis=1)

In [13]:
products1['full_name'] = products1.apply(lambda x: ' '.join(list(dict.fromkeys((x['name_1c_new']+' '+x['ozon_name_new']+' '+x['name_new']+' '+x['wb_name_new']).split()))), axis=1)

In [14]:
products1['article'] = products1['article'].apply(lambda x: x.lower())

In [15]:
products_final = products1.loc[:,['id','article','cost','recommended_price', 'full_name', 'dimension', 'quantity']]

In [16]:
products_final['recommended_price'] = products_final['recommended_price'].fillna(products_final['recommended_price'].median())

In [17]:
products_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   article            496 non-null    object 
 2   cost               491 non-null    float64
 3   recommended_price  496 non-null    float64
 4   full_name          496 non-null    object 
 5   dimension          481 non-null    object 
 6   quantity           496 non-null    float64
dtypes: float64(3), int64(1), object(3)
memory usage: 27.2+ KB


In [18]:
def find_product_id_by_article(article):
    if type(article) == float and pd.isna(article):
        return np.nan
    else:
        #print(f'Артикул {article}, будет поиск')
        try:
            id = int(list(products_final.loc[products_final['article']==article].id)[0]) #fr"'.*{article}.*'"
        except:
            return np.nan
        return id

In [19]:
find_product_id_by_article('008-1')

245

**Лемматизация**

In [21]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp_ru = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])

In [22]:
def lemmatizate(text):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    doc_ru = nlp_ru(text)
    return " ".join([token.lemma_ for token in doc_ru])

In [23]:
products_final['full_name'] = products_final['full_name'].apply(lemmatizate)
parser_final['name_new'] = parser_final['name_new'].apply(lemmatizate)

In [24]:
parser_final

,key,price,dealer_id,name_new,article,dimension,quantity
0,546227,233.0,2,средство универсальный universal spray,,л,0.5
1,546408,175.0,2,концентрат multipower для мытьё пол цитрус,,л,1.0
2,546234,285.0,2,средство для чистка люстр universal anti dust,,л,0.5
3,651258,362.0,2,удалитель ржавчина rust remover,023-05,л,0.5
4,546355,205.0,2,средство мыть для баня и сауна multipower wood,,л,1.0
...,...,...,...,...,...,...,...
2996,717999545,1350.0,17,строительный антисептик глубокий проникновение,,кг,9.9
2997,718000129,1282.0,17,огнебиозащита для древесина огнебио prof 1 выс...,,кг,12.0
2998,900996549,294.0,18,отбеливатель для древесина eco 50 готовый состав,,кг,1.0
2999,530170161,751.0,18,строительный антисептик,,кг,5.6


In [25]:
# OHE
encoder = OneHotEncoder()
try:
    new_demensions = encoder.fit_transform(products_final[['dimension']]).toarray()
    products_final_ohe = pd.concat([products_final, pd.DataFrame(data=new_demensions, columns=encoder.categories_[0])], axis=1).drop(['dimension'], axis=1)
except:
    pass
try:
    new_demensions = encoder.transform(parser_final[['dimension']]).toarray()
    parser_final_ohe = pd.concat([parser_final, pd.DataFrame(data=new_demensions, columns=encoder.categories_[0])], axis=1).drop(['dimension'], axis=1)
except:
    pass

In [26]:
base = products_final_ohe.copy().set_index('id')
train = parser_final_ohe.merge(matches, how='right',left_on='key',right_on='key').dropna(subset=['name_new']).drop_duplicates().groupby('key').first().reset_index()
train = train[['price', 'name_new', 'article','quantity', 'кг', 'л', 'шт', np.nan, 'product_id']]

In [27]:
base

,article,cost,recommended_price,full_name,quantity,кг,л,шт,NaN
id,,,,,,,,,
245,008-1,360.00,858.0,антисептик невымываемый для ответственный конс...,1.00,0.0,1.0,0.0,0.0
3,242-12,460.16,1075.0,антигололед 32 готовый состав,12.00,1.0,0.0,0.0,0.0
443,0024-06 с,307.00,644.0,герметик акриловый цвет сосна пакет для шов де...,0.60,0.0,1.0,0.0,0.0
147,305-2,157.73,342.0,кондиционер для бельё королевский ирис crystal...,2.00,0.0,1.0,0.0,0.0
502,0024-7 б,NaN,891.0,герметик акриловой цвет белый,7.00,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
127,152-5,849.78,1985.0,средство для уборка помещение после пожар с де...,5.00,0.0,1.0,0.0,0.0
160,289-1,201.00,436.0,гель для стирка шерсть шёлк и деликатный ткань...,1.00,0.0,1.0,0.0,0.0
74,192-05,108.60,254.0,cредство для чистка гриль и духовка cooky gril...,0.50,0.0,1.0,0.0,0.0


In [28]:
base[base['cost'].isna()]

,article,cost,recommended_price,full_name,quantity,кг,л,шт,NaN
id,,,,,,,,,
502,0024-7 б,NaN,891.0,герметик акриловой цвет белый,7.0,1.0,0.0,0.0,0.0
503,0024-7 о,NaN,891.0,,0.0,0.0,0.0,0.0,1.0
504,w022-05,NaN,891.0,,0.0,0.0,0.0,0.0,1.0
449,0024-06 м12,NaN,891.0,герметик акриловый цвет медовый 0 6 л,12.0,0.0,0.0,1.0,0.0
454,0024-06 о12,NaN,891.0,герметик акриловый цвет сосна пакет 600мл ук орех,12.0,0.0,0.0,1.0,0.0


In [30]:
korpus = pd.concat([products_final['full_name'], parser_final['name_new']])

In [31]:
nltk.download('stopwords')

stopwords=set(nltk_stopwords.words('russian'))
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
count_tf_idf.fit(korpus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\den_m\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TfidfVectorizer(stop_words=['для', 'о', 'быть', 'ему', 'больше', 'как',
                            'нибудь', 'еще', 'когда', 'тот', 'об', 'том', 'там',
                            'себя', 'один', 'мы', 'ее', 'почти', 'этого', 'тем',
                            'надо', 'ну', 'кто', 'хорошо', 'иногда', 'зачем',
                            'никогда', 'хоть', 'над', 'моя', ...])

In [32]:
pd.DataFrame(count_tf_idf.transform(base['full_name']).toarray()).set_index(base.index)

,0,1,2,3,4,5,6,7,8,9,...,792,793,794,795,796,797,798,799,800,801
id,,,,,,,,,,,,,,,,,,,,,
245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.261028,0.0,0.0,0.0,0.0,0.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [34]:
feature_base_tfidf = count_tf_idf.transform(base['full_name'])
feature_train_tfidf = count_tf_idf.transform(train['name_new'])

base_tfidf = pd.DataFrame(feature_base_tfidf.toarray()).set_index(base.index)
#base_tfidf = pd.concat([base_tfidf, base[['article', 'recommended_price','quantity', 'кг', 'л', 'шт', np.nan]]], axis=1)

train_tfidf = pd.DataFrame(feature_train_tfidf.toarray())
train_tfidf['product_id'] = train['product_id']
#train_tfidf = pd.concat([train_tfidf, train[['price', 'article', 'quantity', 'кг', 'л', 'шт', np.nan]]], axis=1)

In [35]:
base_tfidf

,0,1,2,3,4,5,6,7,8,9,...,792,793,794,795,796,797,798,799,800,801
id,,,,,,,,,,,,,,,,,,,,,
245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.261028,0.0,0.0,0.0,0.0,0.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [36]:
base_tfidf.columns = base_tfidf.columns.astype('str')
train_tfidf.columns = train_tfidf.columns.astype('str')

#base_tfidf = base_tfidf.drop(['recommended_price','article',], axis=1)
#train_tfidf = train_tfidf.drop(['price'], axis=1)

In [37]:
train_tfidf

,0,1,2,3,4,5,6,7,8,9,...,793,794,795,796,797,798,799,800,801,product_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,311
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,242
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,397
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281
1598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,259
1599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,290
1600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321


In [38]:
targets = train_tfidf["product_id"]
train_tfidf.drop("product_id", axis=1, inplace=True)

In [39]:
num = list(base_tfidf.columns)
#num.remove('article')
num1 = list(train_tfidf.columns)
#num1.remove('article')

In [40]:
scaler = MinMaxScaler()
base_tfidf_scaled = pd.DataFrame(scaler.fit_transform(base_tfidf[num]), columns=scaler.feature_names_in_).set_index(base_tfidf.index)
train_tfidf_scaled = pd.DataFrame(scaler.transform(train_tfidf[num1]), columns=scaler.feature_names_in_)

In [41]:
train_tfidf_scaled

,0,1,2,3,4,5,6,7,8,9,...,792,793,794,795,796,797,798,799,800,801
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
def accuracy_aggregator(targets, idx, base_index):
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] for r in el[:25]])
    result = 100 * acc / len(idx)
    return result

In [43]:
dims = base_tfidf_scaled.shape[1]
n_cells = 1 # количество центроидов
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [44]:
# подготовка к поиску
idx_l2.train(np.ascontiguousarray(base_tfidf_scaled.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(base_tfidf_scaled.values).astype('float32'))

In [45]:
# создание словаря для нахождения индекса товара в базовом наборе данных
base_index = {k: v for k, v in enumerate(base_tfidf_scaled.index.to_list())}
vecs, idx = idx_l2.search(np.ascontiguousarray(train_tfidf_scaled.values).astype('float32'), 25)

In [46]:
vecs.shape

(1602, 25)

In [47]:
accuracy_aggregator(targets, idx, base_index)

92.57178526841449

In [48]:
vecs[0]

array([3.326249 , 3.326249 , 3.326249 , 4.878796 , 4.878796 , 5.046179 ,
       5.067953 , 5.082083 , 5.1489134, 5.1741123, 5.174651 , 5.24859  ,
       5.3701286, 5.3888254, 5.527322 , 5.527322 , 5.527322 , 5.527322 ,
       5.5495586, 5.6328125, 5.663789 , 5.699666 , 5.718907 , 5.718907 ,
       5.7790956], dtype=float32)

In [49]:
df_for_cb = pd.DataFrame()

# для каждого запроса из обучающего набора данных
for i in range(train_tfidf.shape[0]):

    # берем индексы FAISS для первых пяти рекомендаций

    df = pd.DataFrame(idx[i])
    # берем расстояния от запроса до рекомендаций FAISS
    df = pd.concat([df, pd.DataFrame(vecs[i])], axis=1)
    df.columns = ['idx', 'distance']
    # индекс вектора-запроса для получения его координат
    df['query_idx'] = i
    # индекс вектора рекомендаций экспертов
    df['target_idx'] = targets[i]
    df_for_cb = pd.concat([df_for_cb, df], ignore_index=True)
# функция для получения индекса рекомендаций в базовом наборе товаров по индексу FAISS
def get_base_idx(row):
    return base_index[row['idx']]
# по внутреннему индексу FAISS восстанавливаем индекс рекомендованного товара в базовом наборе
df_for_cb['base_idx'] = df_for_cb.apply(get_base_idx, axis=1)

df_for_cb['target'] = df_for_cb['target_idx'] == df_for_cb['base_idx']
df_for_cb['target'] = df_for_cb['target'].astype('int')

# добавляем координаты векторов-запросов
#df_for_cb = df_for_cb.merge(train_tfidf, how='inner', left_on='query_idx', right_index=True)

# добавляем координаты векторов-правильных ответов FAISS
#df_for_cb = df_for_cb.merge(base_tfidf, how='inner', left_on='base_idx', right_index=True)

In [50]:
df_for_cb

,idx,distance,query_idx,target_idx,base_idx,target
0,23,3.326249,0,311,503,0
1,35,3.326249,0,311,504,0
2,98,3.326249,0,311,436,0
3,67,4.878796,0,311,315,0
4,108,4.878796,0,311,319,0
...,...,...,...,...,...,...
40045,203,8.186083,1601,320,328,0
40046,438,8.216496,1601,320,325,0
40047,440,8.216496,1601,320,326,0
40048,144,8.240921,1601,320,278,0


In [51]:
scaler = MinMaxScaler()
train.columns = train.columns.astype('str')
base.columns = base.columns.astype('str')
base['price'] = base['recommended_price']
base_f = pd.DataFrame(scaler.fit_transform(base.loc[:,['price','quantity','кг','л','шт','nan']]), columns=scaler.feature_names_in_).set_index(base_tfidf.index)
train_f = pd.DataFrame(scaler.transform(train.loc[:,['price','quantity','кг','л','шт','nan']]), columns=scaler.feature_names_in_)
#base_f = base_f.dropna(subset=['price'])
base_f

,price,quantity,кг,л,шт,nan
id,,,,,,
245,0.057276,0.015385,0.0,1.0,0.0,0.0
3,0.071762,0.184615,1.0,0.0,0.0,0.0
443,0.042991,0.009231,0.0,1.0,0.0,0.0
147,0.022830,0.030769,0.0,1.0,0.0,0.0
502,0.059479,0.107692,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
127,0.132510,0.076923,0.0,1.0,0.0,0.0
160,0.029105,0.015385,0.0,1.0,0.0,0.0
74,0.016956,0.007692,0.0,1.0,0.0,0.0


In [52]:
def features_to_siries(list_of_features):
    f=[]
    for c in list_of_features:
        f.append(c)
    return f

In [53]:
train_f['feats'] = train_f.apply(lambda x: features_to_siries([x['price'],x['quantity'],x['кг'],x['л'],x['шт'],x['nan']]), axis=1)
base_f['feats'] = base_f.apply(lambda x: features_to_siries([x['price'],x['quantity'],x['кг'],x['л'],x['шт'],x['nan']]), axis=1)

In [54]:
base_f

,price,quantity,кг,л,шт,nan,feats
id,,,,,,,
245,0.057276,0.015385,0.0,1.0,0.0,0.0,"[0.05727636849132176, 0.015384615384615385, 0...."
3,0.071762,0.184615,1.0,0.0,0.0,0.0,"[0.07176234979973298, 0.18461538461538463, 1.0..."
443,0.042991,0.009231,0.0,1.0,0.0,0.0,"[0.04299065420560748, 0.009230769230769232, 0...."
147,0.022830,0.030769,0.0,1.0,0.0,0.0,"[0.022830440587449933, 0.03076923076923077, 0...."
502,0.059479,0.107692,1.0,0.0,0.0,0.0,"[0.05947930574098798, 0.1076923076923077, 1.0,..."
...,...,...,...,...,...,...,...
127,0.132510,0.076923,0.0,1.0,0.0,0.0,"[0.13251001335113485, 0.07692307692307693, 0.0..."
160,0.029105,0.015385,0.0,1.0,0.0,0.0,"[0.02910547396528705, 0.015384615384615385, 0...."
74,0.016956,0.007692,0.0,1.0,0.0,0.0,"[0.016955941255006674, 0.007692307692307693, 0..."


In [253]:
total_table_alternative = df_for_cb.merge(train_f['feats'], 
                              how='left',
                              left_on='query_idx',
                              right_index=True).\
                        merge(base_f['feats'], 
                              how='left',
                              left_on='base_idx',
                              right_index=True)

In [55]:
total_table = df_for_cb.merge(train.loc[:,['price','article','quantity','кг','л','шт','nan']], 
                              how='left',
                              left_on='query_idx',
                              right_index=True).\
                        merge(base.loc[:,['recommended_price','article','quantity','кг','л','шт','nan']], 
                              how='left',
                              left_on='base_idx',
                              right_index=True)
total_table

,idx,distance,query_idx,target_idx,base_idx,target,price,article_x,quantity_x,кг_x,л_x,шт_x,nan_x,recommended_price,article_y,quantity_y,кг_y,л_y,шт_y,nan_y
0,23,3.326249,0,311,503,0,7250.0,,16.00,1.0,0.0,0.0,0.0,891.0,0024-7 о,0.0,0.0,0.0,0.0,1.0
1,35,3.326249,0,311,504,0,7250.0,,16.00,1.0,0.0,0.0,0.0,891.0,w022-05,0.0,0.0,0.0,0.0,1.0
2,98,3.326249,0,311,436,0,7250.0,,16.00,1.0,0.0,0.0,0.0,600.0,р1 09005,0.0,0.0,0.0,0.0,1.0
3,67,4.878796,0,311,315,0,7250.0,,16.00,1.0,0.0,0.0,0.0,9868.0,063-65и,65.0,1.0,0.0,0.0,0.0
4,108,4.878796,0,311,319,0,7250.0,,16.00,1.0,0.0,0.0,0.0,5756.0,064-65и,65.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40045,203,8.186083,1601,320,328,0,418.0,,0.25,0.0,1.0,0.0,0.0,901.0,058-09,0.9,0.0,1.0,0.0,0.0
40046,438,8.216496,1601,320,325,0,418.0,,0.25,0.0,1.0,0.0,0.0,551.0,055-09,0.9,0.0,1.0,0.0,0.0
40047,440,8.216496,1601,320,326,0,418.0,,0.25,0.0,1.0,0.0,0.0,1086.0,055-2,2.0,0.0,1.0,0.0,0.0
40048,144,8.240921,1601,320,278,0,418.0,,0.25,0.0,1.0,0.0,0.0,6780.0,062-65,65.0,0.0,1.0,0.0,0.0


In [260]:
def article_check(article_x, article_y):
    if article_x == '':
        return 'unknown'
    else:
        return (article_x == article_y) * 1

In [265]:
total_table

,idx,distance,query_idx,target_idx,base_idx,target,price,article_x,quantity_x,кг_x,...,кг_y,л_y,шт_y,nan_y,quantity,kg,l,cnt,nan,diff_price
0,23,3.326249,0,311,503,0,7250.0,,16.00,1.0,...,0.0,0.0,0.0,1.0,0,0,1,1,0,NaN
1,35,3.326249,0,311,504,0,7250.0,,16.00,1.0,...,0.0,0.0,0.0,1.0,0,0,1,1,0,NaN
2,98,3.326249,0,311,436,0,7250.0,,16.00,1.0,...,0.0,0.0,0.0,1.0,0,0,1,1,0,6650.0
3,67,4.878796,0,311,315,0,7250.0,,16.00,1.0,...,1.0,0.0,0.0,0.0,0,1,1,1,1,2618.0
4,108,4.878796,0,311,319,0,7250.0,,16.00,1.0,...,1.0,0.0,0.0,0.0,0,1,1,1,1,1494.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40045,203,8.186083,1601,320,328,0,418.0,,0.25,0.0,...,0.0,1.0,0.0,0.0,0,1,1,1,1,483.0
40046,438,8.216496,1601,320,325,0,418.0,,0.25,0.0,...,0.0,1.0,0.0,0.0,0,1,1,1,1,133.0
40047,440,8.216496,1601,320,326,0,418.0,,0.25,0.0,...,0.0,1.0,0.0,0.0,0,1,1,1,1,668.0
40048,144,8.240921,1601,320,278,0,418.0,,0.25,0.0,...,0.0,1.0,0.0,0.0,0,1,1,1,1,6362.0


In [56]:
total_table['quantity'] = (total_table['quantity_x'] == total_table['quantity_y']) * 1
total_table['kg'] = (total_table['кг_x'] == total_table['кг_y']) * 1
total_table['l'] = (total_table['л_x'] == total_table['л_y']) * 1
total_table['cnt'] = (total_table['шт_x'] == total_table['шт_y']) * 1
total_table['nan'] = (total_table['nan_x'] == total_table['nan_y']) * 1
total_table['diff_price'] = np.abs(total_table['price'] - total_table['recommended_price'])
#total_table['article'] = total_table['article_x'].apply(lambda x: 'unknown' if type(x['article_x']) == '' else ((x['article_x'] == x['article_y']) * 1))
total_table['article'] = total_table['article_x'].isna()
for i in range(total_table.shape[0]):
    if total_table['article'][i] == True:
        total_table['article'][i]='unknown'
    else:
        total_table['article'][i] = (total_table['article_x'][i] == total_table['article_y'][i]) * 1

C:\Users\den_m\AppData\Local\Temp\ipykernel_1296\1845134057.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_table['article'][i] = (total_table['article_x'][i] == total_table['article_y'][i]) * 1


In [63]:
total_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40050 entries, 0 to 40049
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   idx         40050 non-null  int64  
 1   distance    40050 non-null  float64
 2   query_idx   40050 non-null  int64  
 3   target_idx  40050 non-null  int64  
 4   base_idx    40050 non-null  int64  
 5   target      40050 non-null  int32  
 6   quantity    40050 non-null  int32  
 7   kg          40050 non-null  int32  
 8   l           40050 non-null  int32  
 9   cnt         40050 non-null  int32  
 10  nan         40050 non-null  int32  
 11  diff_price  40050 non-null  float64
 12  0           40050 non-null  float64
 13  1           40050 non-null  float64
dtypes: float64(4), int32(6), int64(4)
memory usage: 3.4 MB


In [58]:
total_table = total_table.drop(['article_x', 'article_y', 'quantity_x',
              'quantity_y', 'кг_x', 'кг_y', 'л_x',
              'л_y', 'шт_x', 'шт_y', 'nan_x', 'nan_y',
              'recommended_price', 'price'], axis=1)

In [60]:
total_scaler = MinMaxScaler()
total_table[['distance', 'diff_price']] = total_scaler.fit_transform(total_table[['distance', 'diff_price']])

In [61]:
total_encoder = OneHotEncoder()

dem = total_encoder.fit_transform(total_table[['article']]).toarray()
total_table = pd.concat([total_table, pd.DataFrame(data=dem, columns=total_encoder.categories_[0])], axis=1).drop(['article'], axis=1)

In [62]:
total_table = total_table.rename({"0":"art_0","1":"art_1", "unknown":"art_unknown"}, axis='columns')

In [64]:
total_table.to_csv('tt_den.csv')